<a href="https://colab.research.google.com/github/juniperlovato/AASP/blob/main/AI_Art_Paper_Ordinal_to_Binary_Cochran_Armitage_5P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#imports

In [35]:
import numpy as np
import pandas as pd
from google.colab import files
import requests
import io
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats.contingency import odds_ratio
# from statsmodels.graphics.mosaicplot import mosaic #library for mosaic
import matplotlib.pyplot as plt
from statsmodels.miscmodels.ordinal_model import OrderedModel

#data

In [65]:
#load data thats cleand ai_art_surveydata_cleaned.csv
Survey =  pd.read_csv('ai_art_surveydata_cleaned.csv')

In [37]:
Ordinal_Binary_variables = Survey[['POC', 'Professional_artist', 'Purchase_art', 'Used_AI_art_models',
                                        'Styleof_ownedby_AI_user', 'Styleof_ownedby_OG_artist', 'Styleof_ownedby_AI_company', 'Required_disclosure',
                                        'AI_pos_development_4art', 'AI_threat_2art_workers']]

#making string numeric values

In [38]:
#dropping nans
Ordinal_Binary_variables = Ordinal_Binary_variables.dropna()

In [39]:
# dropping answers that == 'I’m not sure'
Ordinal_Binary_variables = Ordinal_Binary_variables[Ordinal_Binary_variables["Purchase_art"].str.contains("I’m not sure") == False]
Ordinal_Binary_variables['Purchase_art'].value_counts()

Purchase_art
Yes     395
No       53
Name: count, dtype: int64

In [40]:
#turning to numberic values

Ordinal_Binary_variables['Purchase_art'] = np.where(Ordinal_Binary_variables['Purchase_art'] == 'Yes ', 1, Ordinal_Binary_variables['Purchase_art'])
Ordinal_Binary_variables['Purchase_art'] = np.where(Ordinal_Binary_variables['Purchase_art'] == 'No ', 0, Ordinal_Binary_variables['Purchase_art'])
Ordinal_Binary_variables['Purchase_art'].value_counts()

Purchase_art
1    395
0     53
Name: count, dtype: int64

In [41]:
Ordinal_Binary_variables['Used_AI_art_models'].value_counts()

Used_AI_art_models
No                   219
Yes                  219
I don't know           8
None of the above      2
Name: count, dtype: int64

In [42]:
# getting rid of answers I’m not sure, none of the above
Ordinal_Binary_variables = Ordinal_Binary_variables[Ordinal_Binary_variables["Used_AI_art_models"].str.contains("I don't know") == False]
Ordinal_Binary_variables = Ordinal_Binary_variables[Ordinal_Binary_variables["Used_AI_art_models"].str.contains("None of the above") == False]
Ordinal_Binary_variables['Used_AI_art_models'].unique()

array(['No', 'Yes '], dtype=object)

In [43]:
#turning to numberic values

Ordinal_Binary_variables['Used_AI_art_models'] = np.where(Ordinal_Binary_variables['Used_AI_art_models'] == 'Yes ', 1, Ordinal_Binary_variables['Used_AI_art_models'])
Ordinal_Binary_variables['Used_AI_art_models'] = np.where(Ordinal_Binary_variables['Used_AI_art_models'] == 'No', 0, Ordinal_Binary_variables['Used_AI_art_models'])
Ordinal_Binary_variables['Used_AI_art_models'].value_counts()

Used_AI_art_models
0    219
1    219
Name: count, dtype: int64

In [44]:
# getting rid of answers PNTR
Ordinal_Binary_variables = Ordinal_Binary_variables[Ordinal_Binary_variables["Professional_artist"].str.contains("Prefer not to respond") == False]
Ordinal_Binary_variables['Professional_artist'].unique()

array(['No', 'Yes'], dtype=object)

In [45]:
#turning to numberic values
Ordinal_Binary_variables['Professional_artist'] = np.where(Ordinal_Binary_variables['Professional_artist'] == 'Yes', 1, Ordinal_Binary_variables['Professional_artist'])
Ordinal_Binary_variables['Professional_artist'] = np.where(Ordinal_Binary_variables['Professional_artist'] == 'No', 0, Ordinal_Binary_variables['Professional_artist'])
Ordinal_Binary_variables['Professional_artist'].value_counts()

Professional_artist
0    229
1    198
Name: count, dtype: int64

In [46]:

Ordinal_Binary_variables['POC'].unique()

array(['No ', 'Yes '], dtype=object)

In [47]:
#turning to numberic values
Ordinal_Binary_variables['POC'] = np.where(Ordinal_Binary_variables['POC'] == 'Yes ', 1, Ordinal_Binary_variables['POC'])
Ordinal_Binary_variables['POC'] = np.where(Ordinal_Binary_variables['POC'] == 'No ', 0, Ordinal_Binary_variables['POC'])
Ordinal_Binary_variables['POC'].value_counts()

POC
0    335
1     92
Name: count, dtype: int64

In [48]:
Ordinal_Binary_variables_results = pd.DataFrame()

In [49]:
#selecting the columns that I want for analysis

Ordinal_Binary_variables_results['variable_combo'] = ['Styleof_owner_AIUser_POC', 'Styleof_owner_AIUser_Profartist', 'Styleof_owner_AIUser_Purch', 'Styleof_owner_AIUser_UsedAI',
                                                           'Styleof_owner_OGArtist_POC', 'Styleof_owner_OGArtist_Profartist','Styleof_owner_OGArtist_Purch','Styleof_owner_OGArtist_UsedAI',
                                                           'Styleof_owner_AIComp_POC','Styleof_owner_AIComp_Profartist','Styleof_owner_AIComp_Purch','Styleof_owner_AIComp_UsedAI',
                                                           'Required_disclosure_POC','Required_disclosure_Profartist','Required_disclosure_Purch','Required_disclosure_UsedAI',
                                                           'AI_Pos_POC','AI_Pos_Profartist','AI_Pos_Purch','AI_Pos_UsedAI',
                                                           'AI_Threat_POC','AI_Threat_Profartist','AI_Threat_Purch','AI_Threat_UsedAI',
                                                           ]
# Ordinal_Binary_variables_results

#Making contingency tables

In [50]:
Ordinal_Binary_variables_results['contingency_table'] = ""
# Ordinal_Binary_variables_results

In [51]:
#making a contingency table
Styleof_owner_AIUser_Profartist = pd.crosstab(Ordinal_Binary_variables['Professional_artist'], Ordinal_Binary_variables['Styleof_ownedby_AI_user'], dropna=False, normalize='index')
Styleof_owner_AIUser_Profartist

Styleof_ownedby_AI_user,1.0,2.0,3.0,4.0,5.0
Professional_artist,,,,,
0,0.161572,0.227074,0.231441,0.227074,0.152838
1,0.232323,0.191919,0.166667,0.181818,0.227273


In [52]:

#make a contingency table out of our variables

Styleof_owner_AIUser_POC = pd.crosstab(Ordinal_Binary_variables['POC'], Ordinal_Binary_variables['Styleof_ownedby_AI_user'], dropna=False, normalize='index')
Styleof_owner_AIUser_Profartist = pd.crosstab(Ordinal_Binary_variables['Professional_artist'], Ordinal_Binary_variables['Styleof_ownedby_AI_user'], dropna=False, normalize='index')
Styleof_owner_AIUser_Purch = pd.crosstab(Ordinal_Binary_variables['Purchase_art'], Ordinal_Binary_variables['Styleof_ownedby_AI_user'], dropna=False, normalize='index')
Styleof_owner_AIUser_UsedAI = pd.crosstab(Ordinal_Binary_variables['Used_AI_art_models'], Ordinal_Binary_variables['Styleof_ownedby_AI_user'], dropna=False, normalize='index')
Styleof_owner_OGArtist_POC = pd.crosstab(Ordinal_Binary_variables['POC'], Ordinal_Binary_variables['Styleof_ownedby_OG_artist'], dropna=False, normalize='index')
Styleof_owner_OGArtist_Profartist = pd.crosstab(Ordinal_Binary_variables['Professional_artist'], Ordinal_Binary_variables['Styleof_ownedby_OG_artist'], dropna=False, normalize='index')
Styleof_owner_OGArtist_Purch = pd.crosstab(Ordinal_Binary_variables['Purchase_art'], Ordinal_Binary_variables['Styleof_ownedby_OG_artist'], dropna=False, normalize='index')
Styleof_owner_OGArtist_UsedAI = pd.crosstab(Ordinal_Binary_variables['Used_AI_art_models'], Ordinal_Binary_variables['Styleof_ownedby_OG_artist'], dropna=False, normalize='index')
Styleof_owner_AIComp_POC = pd.crosstab(Ordinal_Binary_variables['POC'], Ordinal_Binary_variables['Styleof_ownedby_AI_company'], dropna=False, normalize='index')
Styleof_owner_AIComp_Profartist = pd.crosstab(Ordinal_Binary_variables['Professional_artist'], Ordinal_Binary_variables['Styleof_ownedby_AI_company'], dropna=False, normalize='index')
Styleof_owner_AIComp_Purch=pd.crosstab(Ordinal_Binary_variables['Purchase_art'], Ordinal_Binary_variables['Styleof_ownedby_AI_company'], dropna=False, normalize='index')
Styleof_owner_AIComp_UsedAI=pd.crosstab(Ordinal_Binary_variables['Used_AI_art_models'], Ordinal_Binary_variables['Styleof_ownedby_AI_company'], dropna=False, normalize='index')
Required_disclosure_POC =  pd.crosstab(Ordinal_Binary_variables['POC'], Ordinal_Binary_variables['Required_disclosure'], dropna=False, normalize='index')
Required_disclosure_Profartist =pd.crosstab(Ordinal_Binary_variables['Professional_artist'], Ordinal_Binary_variables['Required_disclosure'], dropna=False, normalize='index')
Required_disclosure_Purch =pd.crosstab(Ordinal_Binary_variables['Purchase_art'], Ordinal_Binary_variables['Required_disclosure'], dropna=False, normalize='index')
Required_disclosure_UsedAI =pd.crosstab(Ordinal_Binary_variables['Used_AI_art_models'], Ordinal_Binary_variables['Required_disclosure'], dropna=False, normalize='index')
AI_Pos_POC= pd.crosstab(Ordinal_Binary_variables['POC'], Ordinal_Binary_variables['AI_pos_development_4art'], dropna=False, normalize='index')
AI_Pos_Profartist=pd.crosstab(Ordinal_Binary_variables['Professional_artist'], Ordinal_Binary_variables['AI_pos_development_4art'], dropna=False, normalize='index')
AI_Pos_Purch=pd.crosstab(Ordinal_Binary_variables['Purchase_art'], Ordinal_Binary_variables['AI_pos_development_4art'], dropna=False, normalize='index')
AI_Pos_UsedAI=pd.crosstab(Ordinal_Binary_variables['Used_AI_art_models'], Ordinal_Binary_variables['AI_pos_development_4art'], dropna=False, normalize='index')
AI_Threat_POC=pd.crosstab(Ordinal_Binary_variables['POC'], Ordinal_Binary_variables['AI_threat_2art_workers'], dropna=False, normalize='index')
AI_Threat_Profartist=pd.crosstab(Ordinal_Binary_variables['Professional_artist'], Ordinal_Binary_variables['AI_threat_2art_workers'], dropna=False, normalize='index')
AI_Threat_Purch=pd.crosstab(Ordinal_Binary_variables['Purchase_art'], Ordinal_Binary_variables['AI_threat_2art_workers'], dropna=False, normalize='index')
AI_Threat_UsedAI=pd.crosstab(Ordinal_Binary_variables['Used_AI_art_models'], Ordinal_Binary_variables['AI_threat_2art_workers'], dropna=False, normalize='index')

#I want to save these as an array for the data frame later on
Styleof_owner_AIUser_POC_contingency = Styleof_owner_AIUser_POC.values
Styleof_owner_AIUser_Profartist_contingency = Styleof_owner_AIUser_Profartist.values
Styleof_owner_AIUser_Purch_contingency = Styleof_owner_AIUser_Purch.values
Styleof_owner_AIUser_UsedAI_contingency = Styleof_owner_AIUser_UsedAI.values
Styleof_owner_OGArtist_POC_contingency = Styleof_owner_OGArtist_POC.values
Styleof_owner_OGArtist_Profartist_contingency = Styleof_owner_OGArtist_Profartist.values
Styleof_owner_OGArtist_Purch_contingency = Styleof_owner_OGArtist_Purch.values
Styleof_owner_OGArtist_UsedAI_contingency = Styleof_owner_OGArtist_UsedAI.values
Styleof_owner_AIComp_POC_contingency = Styleof_owner_AIComp_POC.values
Styleof_owner_AIComp_Profartist_contingency = Styleof_owner_AIComp_Profartist.values
Styleof_owner_AIComp_Purch_contingency = Styleof_owner_AIComp_Purch.values
Styleof_owner_AIComp_UsedAI_contingency = Styleof_owner_AIComp_UsedAI.values
AI_Pos_POC_contingency = AI_Pos_POC.values
AI_Pos_Profartist_contingency = AI_Pos_Profartist.values
AI_Pos_Purch_contingency = AI_Pos_Purch.values
AI_Pos_UsedAI_contingency = AI_Pos_UsedAI.values
AI_Threat_POC_contingency = AI_Threat_POC.values
AI_Threat_Profartist_contingency = AI_Threat_Profartist.values
AI_Threat_Purch_contingency = AI_Threat_Purch.values
AI_Threat_UsedAI_contingency = AI_Threat_UsedAI.values
Required_disclosure_POC_contingency =Required_disclosure_POC.values
Required_disclosure_Profartist_contingency =Required_disclosure_Profartist.values
Required_disclosure_Purch_contingency = Required_disclosure_Purch.values
Required_disclosure_UsedAI_contingency =Required_disclosure_UsedAI.values

In [53]:
AI_Threat_Profartist

AI_threat_2art_workers,1.0,2.0,3.0,4.0,5.0
Professional_artist,,,,,
0,0.052402,0.135371,0.152838,0.388646,0.270742
1,0.080808,0.161616,0.166667,0.282828,0.308081


In [54]:
#making an empty column for contingency tables
Ordinal_Binary_variables_results['contingency_table'] = ''

In [55]:
#saving all the contingency tables to my new column
Ordinal_Binary_variables_results['contingency_table'] = [Styleof_owner_AIUser_POC_contingency, Styleof_owner_AIUser_Profartist_contingency, Styleof_owner_AIUser_Purch_contingency, Styleof_owner_AIUser_UsedAI_contingency,
Styleof_owner_OGArtist_POC_contingency, Styleof_owner_OGArtist_Profartist_contingency,Styleof_owner_OGArtist_Purch_contingency,Styleof_owner_OGArtist_UsedAI_contingency,
Styleof_owner_AIComp_POC_contingency,Styleof_owner_AIComp_Profartist_contingency,Styleof_owner_AIComp_Purch_contingency,Styleof_owner_AIComp_UsedAI_contingency,
Required_disclosure_POC_contingency,Required_disclosure_Profartist_contingency,Required_disclosure_Purch_contingency,Required_disclosure_UsedAI_contingency,
AI_Pos_POC_contingency,AI_Pos_Profartist_contingency,AI_Pos_Purch_contingency,AI_Pos_UsedAI_contingency,
AI_Threat_POC_contingency,AI_Threat_Profartist_contingency,AI_Threat_Purch_contingency,AI_Threat_UsedAI_contingency,
]

Ordinal_Binary_variables_results.head()


,variable_combo,contingency_table
0,Styleof_owner_AIUser_POC,"[[0.22388059701492538, 0.2, 0.2179104477611940..."
1,Styleof_owner_AIUser_Profartist,"[[0.1615720524017467, 0.22707423580786026, 0.2..."
2,Styleof_owner_AIUser_Purch,"[[0.08163265306122448, 0.22448979591836735, 0...."
3,Styleof_owner_AIUser_UsedAI,"[[0.23222748815165878, 0.27488151658767773, 0...."
4,Styleof_owner_OGArtist_POC,"[[0.10746268656716418, 0.28059701492537314, 0...."


In [56]:
Ordinal_Binary_variables['Styleof_ownedby_AI_user'] = Ordinal_Binary_variables['Styleof_ownedby_AI_user'].astype(int)

#Running cochran armitage test

In [57]:
#getting the cochran armitage test to work on one then ill build a function
data = Ordinal_Binary_variables[["POC", "AI_threat_2art_workers"]]
table = sm.stats.Table.from_data(data, shift_zeros=False)
print(table)
rslt = table.test_ordinal_association()
print(rslt)

A 2x5 contingency table with counts:
[[ 22.  53.  54. 113.  93.]
 [  6.  10.  14.  32.  30.]]
null_mean   242.60421545667447
null_sd     10.402987117878387
pvalue      0.2733265979520707
statistic   254.0
zscore      1.0954338801151586


In [58]:
Ordinal_Binary_variables_results.head()


,variable_combo,contingency_table
0,Styleof_owner_AIUser_POC,"[[0.22388059701492538, 0.2, 0.2179104477611940..."
1,Styleof_owner_AIUser_Profartist,"[[0.1615720524017467, 0.22707423580786026, 0.2..."
2,Styleof_owner_AIUser_Purch,"[[0.08163265306122448, 0.22448979591836735, 0...."
3,Styleof_owner_AIUser_UsedAI,"[[0.23222748815165878, 0.27488151658767773, 0...."
4,Styleof_owner_OGArtist_POC,"[[0.10746268656716418, 0.28059701492537314, 0...."


In [59]:
#making my function
def cochran_armitage(x,y,z):
  data = x[[y,z]]
  table = sm.stats.Table.from_data(data, shift_zeros=False)
  rslt = table.test_ordinal_association()
  return rslt.pvalue, rslt.statistic, rslt.null_mean, rslt.null_sd, rslt.zscore

In [60]:
#lets run my function on all my data now
Styleof_owner_AIUser_POC_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'POC','Styleof_ownedby_AI_user')
Styleof_owner_AIUser_Profartist_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Professional_artist','Styleof_ownedby_AI_user')
Styleof_owner_AIUser_Purch_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Purchase_art','Styleof_ownedby_AI_user')
Styleof_owner_AIUser_UsedAI_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Used_AI_art_models','Styleof_ownedby_AI_user')

Styleof_owner_OGArtist_POC_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'POC','Styleof_ownedby_OG_artist')
Styleof_owner_OGArtist_Profartist_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Professional_artist','Styleof_ownedby_OG_artist')
Styleof_owner_OGArtist_Purch_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Purchase_art','Styleof_ownedby_OG_artist')
Styleof_owner_OGArtist_UsedAI_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Used_AI_art_models','Styleof_ownedby_OG_artist')

Styleof_owner_AIComp_POC_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'POC','Styleof_ownedby_AI_company')
Styleof_owner_AIComp_Profartist_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Professional_artist','Styleof_ownedby_AI_company')
Styleof_owner_AIComp_Purch_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Purchase_art','Styleof_ownedby_AI_company')
Styleof_owner_AIComp_UsedAI_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Used_AI_art_models','Styleof_ownedby_AI_company')

Required_disclosure_POC_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'POC','Required_disclosure')
Required_disclosure_Profartist_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Professional_artist','Required_disclosure')
Required_disclosure_Purch_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Purchase_art','Required_disclosure')
Required_disclosure_UsedAI_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Used_AI_art_models','Required_disclosure')

AI_Pos_POC_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'POC','AI_pos_development_4art')
AI_Pos_Profartist_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Professional_artist','AI_pos_development_4art')
AI_Pos_Purch_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Purchase_art','AI_pos_development_4art')
AI_Pos_UsedAI_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Used_AI_art_models','AI_pos_development_4art')

AI_Threat_POC_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'POC','AI_threat_2art_workers')
AI_Threat_Profartist_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Professional_artist','AI_threat_2art_workers')
AI_Threat_Purch_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Purchase_art','AI_threat_2art_workers')
AI_Threat_UsedAI_Pvalue = rslt.pvalue = cochran_armitage(Ordinal_Binary_variables, 'Used_AI_art_models','AI_threat_2art_workers')


In [61]:
#making an empty column to storre the results
Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'] = ''
#and adding my results to that new columns
Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'] = [Styleof_owner_AIUser_POC_Pvalue, Styleof_owner_AIUser_Profartist_Pvalue, Styleof_owner_AIUser_Purch_Pvalue, Styleof_owner_AIUser_UsedAI_Pvalue,
Styleof_owner_OGArtist_POC_Pvalue, Styleof_owner_OGArtist_Profartist_Pvalue,Styleof_owner_OGArtist_Purch_Pvalue,Styleof_owner_OGArtist_UsedAI_Pvalue,
Styleof_owner_AIComp_POC_Pvalue,Styleof_owner_AIComp_Profartist_Pvalue,Styleof_owner_AIComp_Purch_Pvalue,Styleof_owner_AIComp_UsedAI_Pvalue,
Required_disclosure_POC_Pvalue,Required_disclosure_Profartist_Pvalue,Required_disclosure_Purch_Pvalue,Required_disclosure_UsedAI_Pvalue,
AI_Pos_POC_Pvalue,AI_Pos_Profartist_Pvalue,AI_Pos_Purch_Pvalue,AI_Pos_UsedAI_Pvalue,
AI_Threat_POC_Pvalue,AI_Threat_Profartist_Pvalue,AI_Threat_Purch_Pvalue,AI_Threat_UsedAI_Pvalue,
]
Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'] = Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'].astype(str)


In [62]:
Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'] = Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'].str.replace(r"\(|\)", "")
Ordinal_Binary_variables_results.head()

,variable_combo,contingency_table,pvalue_statistic_null_mean_null_sd_zscore
0,Styleof_owner_AIUser_POC,"[[0.22388059701492538, 0.2, 0.2179104477611940...","(0.0074644164626657935, 214.0, 182.27634660421..."
1,Styleof_owner_AIUser_Profartist,"[[0.1615720524017467, 0.22707423580786026, 0.2...","(0.9838908385494967, 392.0, 392.2903981264637,..."
2,Styleof_owner_AIUser_Purch,"[[0.08163265306122448, 0.22448979591836735, 0....","(0.15992987280839666, 736.0, 748.9180327868852..."
3,Styleof_owner_AIUser_UsedAI,"[[0.23222748815165878, 0.27488151658767773, 0....","(1.2292081366740568e-05, 491.0, 427.9531615925..."
4,Styleof_owner_OGArtist_POC,"[[0.10746268656716418, 0.28059701492537314, 0....","(0.00037047083566542723, 237.0, 198.4355971896..."


In [63]:
Ordinal_Binary_variables_results['Pvalue'] = Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'].str.split(',').str[0]
Ordinal_Binary_variables_results['statistic'] = Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'].str.split(',').str[1]
Ordinal_Binary_variables_results['null_mean'] = Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'].str.split(',').str[2]
Ordinal_Binary_variables_results['null_sd'] = Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'].str.split(',').str[3]
Ordinal_Binary_variables_results['zscore'] = Ordinal_Binary_variables_results['pvalue_statistic_null_mean_null_sd_zscore'].str.split(',').str[4]

Ordinal_Binary_variables_results = Ordinal_Binary_variables_results.drop(columns=['pvalue_statistic_null_mean_null_sd_zscore'])